# Movies scraping

=========================================================================================================

**AUTHOR**: Mengshan Jin

**CREATION DATE**: 08/20/2017

=========================================================================================================


**PROGRAM DESCRIPTION**: Scraping more movies

**INPUT DATASETS**: 01_Data/Outputs/imdb_storyline_recs.csv

**OUTPUT DATASETS**: 


=========================================================================================================

**PROGRAM CHANGE HISTORY**

Date|Author|Change|
----|------|------|

In [1]:
import urllib2
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [2]:
imdb_storyline_recs = pd.read_csv("../Outputs/imdb_storyline_recs.csv")

In [3]:
imdb_storyline_recs.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,recommendation_3,recommendation_4,recommendation_5,recommendation_6,recommendation_7,recommendation_8,recommendation_9,recommendation_10,recommendation_11,recommendation_12
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,tt0120338,tt0454876,tt1454468,tt3659388,tt0816711,tt1951264,tt0371746,tt0416449,tt0119654,tt0848228
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,tt0325980,tt0903624,tt1170358,tt2310332,tt0121766,tt1201607,tt0371746,tt0121765,tt1300854,tt0800369
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,tt0381061,tt2381249,tt0113189,tt1229238,tt0143145,tt0246460,tt0117060,tt0120347,tt0317919,tt0097742
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,tt1375666,tt0816692,tt0482571,tt0167261,tt0120737,tt1853728,tt0167260,tt0086190,tt0137523,tt0172495
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
imdb_storyline_recs.columns

Index([u'color', u'director_name', u'num_critic_for_reviews', u'duration',
       u'director_facebook_likes', u'actor_3_facebook_likes', u'actor_2_name',
       u'actor_1_facebook_likes', u'gross', u'genres', u'actor_1_name',
       u'movie_title', u'num_voted_users', u'cast_total_facebook_likes',
       u'actor_3_name', u'facenumber_in_poster', u'plot_keywords',
       u'movie_imdb_link', u'num_user_for_reviews', u'language', u'country',
       u'content_rating', u'budget', u'title_year', u'actor_2_facebook_likes',
       u'imdb_score', u'aspect_ratio', u'movie_facebook_likes', u'storyline',
       u'movie_imdb_id', u'recommendation_1', u'recommendation_2',
       u'recommendation_3', u'recommendation_4', u'recommendation_5',
       u'recommendation_6', u'recommendation_7', u'recommendation_8',
       u'recommendation_9', u'recommendation_10', u'recommendation_11',
       u'recommendation_12'],
      dtype='object')

## Algorithm:
for id in movie_imdb_id:    
&nbsp;&nbsp;&nbsp;&nbsp;for rec in recommendations:    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;if rec in movie_imdb_id:    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;copy paste    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;else:    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;scrape

In [5]:
def scrape_metascore(movie_id):
    link = 'http://www.imdb.com/title/'+movie_id+'/criticreviews?ref_=tt_ql_op_6'
    try:
        html = urllib2.urlopen(link)
        soup = BeautifulSoup(html, "lxml")
        block = soup.findAll("div", {"class":"metascore_block"})[0]
        metascore = block.findAll("span")[0].text
        # metascore = block.find("div", {"class":"metascore_wrap"})[0].find('span').text
        num_critic_for_reviews = block.findAll("span")[1].text
        return {"metascore":metascore, "num_critic_for_reviews":num_critic_for_reviews}
    except IndexError:
        return {"metascore":np.nan, "num_critic_for_reviews":np.nan}

In [6]:
def scrape_imdb(movie_id):
    link = 'http://www.imdb.com/title/'+movie_id+'/?ref_=fn_tt_tt_1'
    html = urllib2.urlopen(link)
    soup = BeautifulSoup(html, "lxml")
    res_dict = {"movie_imdb_link":link, "movie_imdb_id":movie_id}
    try:
        titleNyear = soup.findAll("div", {"class":"title_wrapper"})[0].find("h1").text
        movie_title = re.sub(r'\xa0 ', '', re.sub(r'\([^)]*\)', '', titleNyear))
        title_year = re.search(r'\((.*?)\)', titleNyear).group(1)
        res_dict.update({"movie_title":movie_title, "title_year":title_year})
    except IndexError:
        res_dict.update({"movie_title":np.nan, "title_year":np.nan})
    except AttributeError:
        res_dict.update({"movie_title":movie_title, "title_year":np.nan})
    try:
        genresAll = soup.findAll("div", {"itemprop":"genre"})[0].findAll("a")
        genres = "|".join([item.text.strip() for item in genresAll])
        res_dict.update({"genres":genres})
    except IndexError:
        res_dict.update({"genres":np.nan})
    try:
        desc = soup.findAll("div", {"class":"inline canwrap"})[0].find('p').text
        storyline = re.split(r'\s{2,}', desc)[0].strip()
        res_dict.update({"storyline":storyline})
    except IndexError:
        res_dict.update({"storyline":np.nan})
    try:
        imdb_score = soup.findAll("span", {"itemprop":"ratingValue"})[0].text
        res_dict.update({"imdb_score":imdb_score})
    except IndexError:
        res_dict.update({"imdb_score":np.nan})
    return res_dict

# Whenever possible, avoid explicit for loops!!!

In [ ]:
# Algorithm:
# results = []
# for i in range(imdb_storyline_recs.shape[0]):
#     tmp = {'num_critic_for_reviews': imdb_storyline_recs.loc[i, 'num_critic_for_reviews'],
#            'genres': imdb_storyline_recs.loc[i, 'genres'],
#            'movie_title': imdb_storyline_recs.loc[i, 'movie_title'],
#            'movie_imdb_link': imdb_storyline_recs.loc[i, 'movie_imdb_link'],
#            'title_year': imdb_storyline_recs.loc[i, 'title_year'],
#            'imdb_score': imdb_storyline_recs.loc[i, 'imdb_score'],
#            'metascore': scrape_metascore(imdb_storyline_recs.loc[i, 'movie_imdb_id'])['metascore'],
#            'storyline': imdb_storyline_recs.loc[i, 'storyline'],
#            'movie_imdb_id': imdb_storyline_recs.loc[i, 'movie_imdb_id']}
#     results.append(tmp)
#     for j in range(12):
#         mid = imdb_storyline_recs.loc[i, 'recommendation_'+str(j+1)]
#         if pd.isnull(mid):
#             pass
#         elif mid in imdb_storyline_recs['movie_imdb_id']:
#             pass
#         else:
#             reviews = scrape_metascore(mid)
#             imdb = scrape_imdb(mid)
#             imdb.update(reviews)
#             results.append(imdb)

In [7]:
imdb_selected = imdb_storyline_recs[['num_critic_for_reviews', 'genres', 'movie_title',
                                     'title_year', 'imdb_score', 'storyline', 'movie_imdb_id']]
imdb_selected['metascore'] = imdb_selected['movie_imdb_id'].apply(lambda x: scrape_metascore(x)['metascore'])

HTTPError: HTTP Error 302: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Temporarily

In [ ]:
# save to csv!!!
res_df.to_csv("../Outputs/imdb_new_metadata.csv", index=False)